<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Телеком" data-toc-modified-id="Телеком-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Телеком</a></span><ul class="toc-item"><li><span><a href="#План-работы" data-toc-modified-id="План-работы-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>План работы</a></span></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка</a></span></li><li><span><a href="#Исследовательский-анализ-данных" data-toc-modified-id="Исследовательский-анализ-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Исследовательский анализ данных</a></span></li><li><span><a href="#Построение-модели" data-toc-modified-id="Построение-модели-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Построение модели</a></span></li><li><span><a href="#Отчёт" data-toc-modified-id="Отчёт-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Отчёт</a></span></li></ul></li></ul></div>

# Телеком

## План работы
* Предобработка данных: ознакомиться с данными, объединить датасеты, обработать пропуски, дубликаты и некорректные значения, привести данные к корректным типам, выделить целевой признак, удалить лишние столбцы.
* Исследовательский анализ данных: исследовать категориальные и количественные признаки, баланс классов целевого признака.
* Построение модели: подготовить признаки к обучению, разделить датасет на выборки, обучить модели с различными гиперпараметрами и выбрать финальную модель.
* Тестирование модели: проверить качество финальной модели на тестовой выборке.
* Вывод: проанализировать полученные результаты.

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as warn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, precision_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from imblearn.over_sampling import BorderlineSMOTE

##  Предобработка

**Загрузим данные**

In [33]:
path_local = '/Users/bogda/anaconda3/projects/praktikum/project_project/final_provider/'
path_yandex = '/datasets/final_provider/'

In [34]:
files = ['contract', 'personal', 'internet', 'phone']
data = {}

In [35]:
def tryexept(path):
    for i in files:
        data[i] = pd.read_csv(path + i + '.csv', index_col= 'customerID')
        print('\n' + i)
        display(data[i].sample(5))
        data[i].info()
        print('—' * 54)

In [36]:
try:
    tryexept(path_local)
except FileNotFoundError as e:
    print(e)
    tryexept(path_yandex)


contract


,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
customerID,,,,,,,
5687-DKDTV,2019-07-01,2019-11-01 00:00:00,Month-to-month,No,Mailed check,20.35,77.5
6689-VRRTK,2016-06-01,No,One year,Yes,Credit card (automatic),109.80,4860.35
2371-KFUOG,2015-04-01,No,One year,No,Bank transfer (automatic),99.15,5720.95
1087-UDSIH,2019-08-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,19.60,59.75
5914-DVBWJ,2018-05-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,85.45,1505.85


<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   BeginDate         7043 non-null   object 
 1   EndDate           7043 non-null   object 
 2   Type              7043 non-null   object 
 3   PaperlessBilling  7043 non-null   object 
 4   PaymentMethod     7043 non-null   object 
 5   MonthlyCharges    7043 non-null   float64
 6   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(6)
memory usage: 440.2+ KB
——————————————————————————————————————————————————————

personal


,gender,SeniorCitizen,Partner,Dependents
customerID,,,,
6016-NXBNJ,Male,0,No,No
2680-XKKNJ,Female,0,No,No
3045-XETSH,Female,0,No,No
8443-ZRDBZ,Male,0,No,No
3389-KTRXV,Female,0,Yes,Yes


<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gender         7043 non-null   object
 1   SeniorCitizen  7043 non-null   int64 
 2   Partner        7043 non-null   object
 3   Dependents     7043 non-null   object
dtypes: int64(1), object(3)
memory usage: 275.1+ KB
——————————————————————————————————————————————————————

internet


,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
customerID,,,,,,,
6253-GNHWH,DSL,Yes,Yes,Yes,Yes,No,No
9738-QLWTP,Fiber optic,No,No,Yes,Yes,No,No
4998-IKFSE,Fiber optic,No,No,Yes,No,Yes,Yes
6988-CJEYV,Fiber optic,No,Yes,No,No,Yes,Yes
8165-CBKXO,DSL,No,Yes,Yes,No,Yes,Yes


<class 'pandas.core.frame.DataFrame'>
Index: 5517 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   InternetService   5517 non-null   object
 1   OnlineSecurity    5517 non-null   object
 2   OnlineBackup      5517 non-null   object
 3   DeviceProtection  5517 non-null   object
 4   TechSupport       5517 non-null   object
 5   StreamingTV       5517 non-null   object
 6   StreamingMovies   5517 non-null   object
dtypes: object(7)
memory usage: 344.8+ KB
——————————————————————————————————————————————————————

phone


,MultipleLines
customerID,
9605-WGJVW,No
7996-BPXHY,Yes
0468-YRPXN,No
0870-VEMYL,No
3078-ZKNTS,No


<class 'pandas.core.frame.DataFrame'>
Index: 6361 entries, 5575-GNVDE to 3186-AJIEK
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MultipleLines  6361 non-null   object
dtypes: object(1)
memory usage: 99.4+ KB
——————————————————————————————————————————————————————


**Небольшой обзор, пропусков нет(на первый взгляд:), две последних таблицы содержат меньше данных чем первые, причина в том что клиенты могут пользоваться разными услугами независимо - заполнить пропуски. Не соответсвтие типов данных и столбцов в таблице contract 'BeginDate', 'EndDate' - привести к дате, 'TotalCharges' к float.В таблице personal столбец 'SeniorSitizen' числовой хотя это категоральный признак. Разный регистр, нет целевого признака.**

**Объеденим таблицы в один датасет, приведем названия столбцов к одному регистру**

In [38]:
df = data['contract'].join(data['personal']).join(data['internet']).join(data['phone'])
df.columns = [k.lower() for k in list(df.columns)]
display(df.sample(5))
df.info()

,begindate,enddate,type,paperlessbilling,paymentmethod,monthlycharges,totalcharges,gender,seniorcitizen,partner,dependents,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,multiplelines
customerID,,,,,,,,,,,,,,,,,,,
4112-LUEIZ,2019-03-01,2020-01-01 00:00:00,Month-to-month,Yes,Electronic check,89.50,863.1,Male,0,No,No,Fiber optic,No,No,No,No,Yes,Yes,No
9777-WJJPR,2017-07-01,No,Month-to-month,No,Credit card (automatic),88.65,2683.2,Male,0,Yes,No,Fiber optic,No,No,Yes,No,Yes,No,Yes
2990-IAJSV,2014-02-01,No,Two year,No,Bank transfer (automatic),92.00,6632.75,Male,0,No,No,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Yes
6723-CEGQI,2014-09-01,No,Two year,No,Mailed check,45.25,2933.95,Female,0,No,Yes,DSL,No,Yes,No,Yes,Yes,No,NaN
5110-CHOPY,2015-02-01,No,Two year,No,Electronic check,53.60,3237.05,Female,0,No,No,DSL,Yes,No,Yes,No,Yes,Yes,NaN


<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   begindate         7043 non-null   object 
 1   enddate           7043 non-null   object 
 2   type              7043 non-null   object 
 3   paperlessbilling  7043 non-null   object 
 4   paymentmethod     7043 non-null   object 
 5   monthlycharges    7043 non-null   float64
 6   totalcharges      7043 non-null   object 
 7   gender            7043 non-null   object 
 8   seniorcitizen     7043 non-null   int64  
 9   partner           7043 non-null   object 
 10  dependents        7043 non-null   object 
 11  internetservice   5517 non-null   object 
 12  onlinesecurity    5517 non-null   object 
 13  onlinebackup      5517 non-null   object 
 14  deviceprotection  5517 non-null   object 
 15  techsupport       5517 non-null   object 
 16  streamingtv       5517 non-null 

**Создадим столбец 'outflow' с целевым признаком**

In [39]:
def func(row):
    if row.enddate == "No":
        return 0
    else:
        return 1
df['outflow'] = df.apply(func, axis=1) 

In [40]:
df

,begindate,enddate,type,paperlessbilling,paymentmethod,monthlycharges,totalcharges,gender,seniorcitizen,partner,dependents,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,multiplelines,outflow
customerID,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,Yes,No,DSL,No,Yes,No,No,No,No,NaN,0
5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,Male,0,No,No,DSL,Yes,No,Yes,No,No,No,No,0
3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,DSL,Yes,Yes,No,No,No,No,No,1
7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,No,No,DSL,Yes,No,Yes,Yes,No,No,NaN,0
9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,Fiber optic,No,No,No,No,No,No,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.5,Male,0,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,Yes,0
2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.9,Female,0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Yes,0
4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45,Female,0,Yes,Yes,DSL,Yes,No,No,No,No,No,NaN,0


**Заменим столбцы 'begindate', 'enddate' на столбец 'contract_length' так мы избавимся от утечки целевого признака и сохраним информативность временного признака. Для начала приведём их к дате.**

In [41]:
df['begindate'] = pd.to_datetime(df['begindate'], errors='coerce')
df['enddate'] = pd.to_datetime(df['enddate'], errors='coerce')

**Заменим пропуски на дату предоставления данных**

In [42]:
df['enddate'].fillna(pd.to_datetime('2020-02-01'), inplace=True)

In [43]:
def duration(row):
    return len(pd.date_range(start=row['begindate'], end=row['enddate']))
df['contract_length'] = df.apply(duration, axis=1)
df.drop(['begindate', 'enddate'], axis=1, inplace=True)

In [44]:
df

,type,paperlessbilling,paymentmethod,monthlycharges,totalcharges,gender,seniorcitizen,partner,dependents,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,multiplelines,outflow,contract_length
customerID,,,,,,,,,,,,,,,,,,,
7590-VHVEG,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,Yes,No,DSL,No,Yes,No,No,No,No,NaN,0,32
5575-GNVDE,One year,No,Mailed check,56.95,1889.5,Male,0,No,No,DSL,Yes,No,Yes,No,No,No,No,0,1037
3668-QPYBK,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,DSL,Yes,Yes,No,No,No,No,No,1,62
7795-CFOCW,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,No,No,DSL,Yes,No,Yes,Yes,No,No,NaN,0,1372
9237-HQITU,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,Fiber optic,No,No,No,No,No,No,No,1,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,One year,Yes,Mailed check,84.80,1990.5,Male,0,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,Yes,0,731
2234-XADUH,One year,Yes,Credit card (automatic),103.20,7362.9,Female,0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Yes,0,2192
4801-JZAZL,Month-to-month,Yes,Electronic check,29.60,346.45,Female,0,Yes,Yes,DSL,Yes,No,No,No,No,No,NaN,0,338


**Приведём данные столбца totalcharges к вещественному типу.**

In [45]:
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')

## Исследовательский анализ данных

## Построение модели

## Отчёт